In [ ]:
%%capture
!pip install transformers datasets
!pip install accelerate
!pip install huggingface_hub
!pip install codebleu
!pip install tree-sitter-python

In [ ]:
from datasets import load_dataset

# Load the NL-code-search-Adv dataset from CodeXGLUE
dataset = load_dataset("google/code_x_glue_tc_nl_code_search_adv")

In [ ]:
shuffled_dataset = dataset['test'].shuffle()

In [ ]:
dataset

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

checkpoint = "Salesforce/codet5p-220m"
device = "cuda"# for GPU usage or "cpu" for CPU usage

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = T5ForConditionalGeneration.from_pretrained(checkpoint).to(device)

In [ ]:
shuffled_dataset

In [ ]:
shuffled_dataset.select(range(100))

In [ ]:
def generate_code(description, tokenizer, model):
    inputs = tokenizer.encode("translate English to Python: " + description, return_tensors="pt", max_length=512, truncation=True).to(device)
    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Generate predictions for the dataset
predictions = []
references = []

for example in shuffled_dataset.select(range(1000)):
    description = example['docstring']
    reference_code = example['code']
    
    predicted_code = generate_code(description, tokenizer, model)
    
    predictions.append(predicted_code)
    references.append(reference_code)
with open('references.txt', 'w') as f:
    for ref in references:
        f.write(f"{ref}\n")
with open('predictions.txt', 'w') as f:
    for pred in predictions:
        f.write(f"{pred}\n")

In [ ]:
from codebleu import calc_codebleu
result = calc_codebleu(references, predictions, lang="python", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
print("CodeBLEU Score:", codebleu_score)